In [18]:
import pandas as pd
import sqlite3

In [19]:
gen_cols = [
    'plant_code',
    'plant_name',
    'generator_id',
    'operating_year',
    'nameplate_capacity_mw',
    'summer_capacity_mw',
    'winter_capacity_mw',
    'minimum_load_mw',
    'energy_source_1',
    'technology',
    'status',
    'prime_mover',
    'operating_month',
    'operating_year',
    'planned_retirement_month',
    'planned_retirement_year',
    ]

con = sqlite3.connect("../data/pudl/pudl.sqlite")

cur = con.cursor()

res = cur.execute("""SELECT m.name as tableName,
       p.name as columnName
FROM sqlite_master m
left outer join pragma_table_info((m.name)) p
     on m.name <> p.name
order by tableName, columnName""")

table_cols = res.fetchall()



In [20]:
# table_cols is a list of tuples, where each tuple is a table name and a column name

exact_matches = []
similar_matches = []
not_found = []

for col in gen_cols:
    found_col = False
    for table, column in table_cols:
        if col == column:
            exact_matches.append((col, table))
            found_col = True
        elif column and col not in exact_matches and col in column:
            similar_matches.append((col, table, column))
            found_col = True
    if not found_col:
        not_found.append(col)



In [30]:


print("Exact gen matches:")
pd.Series(exact_matches)

Exact gen matches:


0     (generator_id, _out_eia__monthly_capacity_fact...
1     (generator_id, _out_eia__monthly_derived_gener...
2     (generator_id, _out_eia__monthly_fuel_cost_by_...
3     (generator_id, _out_eia__monthly_heat_rate_by_...
4     (generator_id, _out_eia__yearly_capacity_facto...
5     (generator_id, _out_eia__yearly_derived_genera...
6     (generator_id, _out_eia__yearly_fuel_cost_by_g...
7           (generator_id, _out_eia__yearly_generators)
8     (generator_id, _out_eia__yearly_heat_rate_by_g...
9     (generator_id, core_eia860__assn_boiler_genera...
10          (generator_id, core_eia860__scd_generators)
11           (generator_id, core_eia860__scd_ownership)
12    (generator_id, core_eia860m__changelog_generat...
13      (generator_id, core_eia923__monthly_generation)
14          (generator_id, core_eia__entity_generators)
15           (generator_id, core_epa__assn_eia_epacamd)
16    (generator_id, core_epa__assn_eia_epacamd_subp...
17         (generator_id, out_eia860__yearly_own

In [22]:
print("Similar gen matches:")
pd.Series(similar_matches)

Similar gen matches:


0      (plant_name, _out_eia__monthly_fuel_cost_by_ge...
1      (plant_name, _out_eia__plants_utilities, plant...
2      (plant_name, _out_eia__yearly_fuel_cost_by_gen...
3      (plant_name, _out_eia__yearly_generators, plan...
4      (plant_name, _out_ferc1__yearly_plants_utiliti...
                             ...                        
134    (prime_mover, out_eia__yearly_plant_parts, pri...
135    (prime_mover, out_pudl__yearly_assn_eia_ferc1_...
136    (operating_year, out_eia__yearly_generators_by...
137    (operating_year, out_eia__yearly_plant_parts, ...
138    (operating_year, out_pudl__yearly_assn_eia_fer...
Length: 139, dtype: object

In [31]:
print("Not found gen columns:")
for col in not_found:
    print(col)

Not found gen columns:
plant_code
nameplate_energy_capacity_mwh
maximum_charge_rate_mw
maximum_discharge_rate_mw
storage_technology_1


In [25]:
for table, column in table_cols:
    if 'storage' in table and 'eia860' in table:
        print(table)

In [26]:
storage_cols = [
    'plant_code',
    'generator_id',
    'nameplate_energy_capacity_mwh',
    'maximum_charge_rate_mw',
    'maximum_discharge_rate_mw',
    'storage_technology_1']

exact_matches = []
similar_matches = []
not_found = []

for col in storage_cols:
    found_col = False
    for table, column in table_cols:
        if col == column:
            exact_matches.append((col, table))
            found_col = True
        elif column and col not in exact_matches and col in column:
            similar_matches.append((col, table, column))
            found_col = True
    if not found_col:
        not_found.append(col)

In [28]:

print("\nNot found storage:")
for col in not_found:
    print(col)


Not found storage:
plant_code
nameplate_energy_capacity_mwh
maximum_charge_rate_mw
maximum_discharge_rate_mw
storage_technology_1
